# HRN (Hierarchical Representation Network) - 3D Face Reconstruction
## arXiv:2302.14434v2 - Lei et al., DAMO Academy, Alibaba Group

**목차**
1. 환경 설정 & Google Drive 마운트
2. 데이터 탐색 (DICOM CT + FaceScape)
3. CT 데이터 시각화 및 2D 얼굴 이미지 추출
4. HRN 모델 설치 및 3D 얼굴 복원
5. FaceScape 데이터 실험
6. 최신 기법 비교 (CodeFormer, GFPGAN)
7. 정량적/정성적 결과 분석

> **주의**: 모든 데이터는 Google Drive에서 읽고, 모든 결과는 Google Drive에 저장합니다.

---
## 1. 환경 설정 & Google Drive 마운트

In [ ]:
# ============================================================
# 환경 확인 및 런타임 버전 권장사항
# ★ 이 프로젝트는 Python 3.10~3.11 + PyTorch 2.x 환경에서 테스트되었습니다
# ★ Colab 메뉴: 런타임 → 런타임 유형 변경 → 런타임 버전 = 2025.07
# ============================================================
import sys
import subprocess

py_ver = f'{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}'
print(f'Python: {py_ver}')

# PyTorch/CUDA 확인
try:
    import torch
    pt_ver = torch.__version__
    cuda_ver = torch.version.cuda or 'None'
    cuda_available = torch.cuda.is_available()
    gpu_name = torch.cuda.get_device_name(0) if cuda_available else 'No GPU'
    print(f'PyTorch: {pt_ver}')
    print(f'CUDA 사용 가능: {cuda_available}')
    print(f'CUDA 버전: {cuda_ver}')
    print(f'GPU: {gpu_name}')
except ImportError:
    pt_ver = 'not installed'
    cuda_ver = 'unknown'
    cuda_available = False
    print('PyTorch not installed')

print()

# 호환성 체크
warnings = []

if not cuda_available:
    warnings.append(
        'GPU가 감지되지 않았습니다!\n'
        '   -> 런타임 -> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU (A100 권장)'
    )

if sys.version_info >= (3, 12):
    warnings.append(
        f'Python {py_ver} 감지. ModelScope/basicsr는 Python 3.10~3.11에서 테스트되었습니다.\n'
        f'   -> 런타임 -> 런타임 유형 변경 -> 런타임 버전을 "2025.07"로 변경 권장'
    )

if 'torch' in dir() and hasattr(torch, '__version__'):
    pt_major_minor = tuple(int(x) for x in torch.__version__.split('+')[0].split('.')[:2])
    if pt_major_minor >= (2, 7):
        warnings.append(
            f'PyTorch {pt_ver} 감지. PyTorch3D pre-built wheel은 2.4.x까지만 지원.\n'
            f'   -> 소스 빌드로 설치됩니다 (최초 ~15분 소요, 이후 Drive 캐시에서 < 1분)'
        )

if warnings:
    print('=' * 60)
    print('  환경 경고')
    print('=' * 60)
    for i, w in enumerate(warnings, 1):
        print(f'  [{i}] {w}')
    print('=' * 60)
    print()
    print('현재 환경에서도 실행을 시도하지만, 문제 발생 시:')
    print('  Colab 메뉴: 런타임 -> 런타임 유형 변경 -> 런타임 버전 -> "2025.07"')
else:
    print('=' * 60)
    print('  환경 호환성 확인 완료!')
    print('=' * 60)

In [ ]:
# GPU 확인 (상세)
!nvidia-smi
print()
# ninja 빌드 시스템 사전 확인 (PyTorch3D, nvdiffrast 빌드에 필요)
!which ninja 2>/dev/null && echo 'ninja: OK' || echo 'ninja: Cell 11에서 설치 예정'

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob

DRIVE_ROOT = '/content/drive/MyDrive'

# ============================================================
# Google Drive 구조 탐색 및 경로 자동 설정
# ============================================================
print('=== Google Drive 구조 탐색 중 ===\n')

# 1단계: Drive 루트 내용 출력
print('[1] Drive 루트 내용:')
for item in sorted(os.listdir(DRIVE_ROOT)):
    full = os.path.join(DRIVE_ROOT, item)
    marker = '📁' if os.path.isdir(full) else '📄'
    print(f'  {marker} {item}')

# ============================================================
# 2단계: BISTOOL 루트 탐색
# ============================================================
BISTOOL_ROOT = None
for candidate in [
    os.path.join(DRIVE_ROOT, 'BISTOOL'),
    os.path.join(DRIVE_ROOT, 'Project', 'BISTOOL'),
    os.path.join(DRIVE_ROOT, 'LAB', 'Project', 'BISTOOL'),
]:
    if os.path.isdir(candidate):
        BISTOOL_ROOT = candidate
        break

if BISTOOL_ROOT:
    print(f'\n[2] BISTOOL 루트: {BISTOOL_ROOT}')
    print('    내용:')
    for item in sorted(os.listdir(BISTOOL_ROOT)):
        full = os.path.join(BISTOOL_ROOT, item)
        marker = '📁' if os.path.isdir(full) else '📄'
        print(f'      {marker} {item}')
else:
    print('\n[2] BISTOOL 폴더를 찾지 못했습니다. 위 Drive 루트 내용을 확인하세요.')

# ============================================================
# 3단계: DICOM 데이터 (DATA_ROOT) 탐색
# ============================================================
DATA_ROOT = None
found_zips = []

search_dirs = []
if BISTOOL_ROOT:
    search_dirs.append(os.path.join(BISTOOL_ROOT, 'Data'))
    search_dirs.append(BISTOOL_ROOT)
search_dirs.extend([
    os.path.join(DRIVE_ROOT, 'Data'),
    DRIVE_ROOT,
])

CT_IDS = ['002', '003', '004', '005', '007', '009']

for search_dir in search_dirs:
    if not os.path.isdir(search_dir):
        continue
    try:
        items = os.listdir(search_dir)
    except PermissionError:
        continue
    for item in items:
        if item.endswith('.zip') and item.replace('.zip', '') in CT_IDS:
            found_zips.append(os.path.join(search_dir, item))
            if DATA_ROOT is None:
                DATA_ROOT = search_dir

if found_zips:
    print(f'\n[3] DICOM zip 파일 발견:')
    for z in found_zips:
        size_mb = os.path.getsize(z) / (1024**2)
        print(f'      {z} ({size_mb:.0f} MB)')

if DATA_ROOT:
    print(f'\n✅ DICOM 데이터 루트: {DATA_ROOT}')
else:
    print(f'\n⚠️ DICOM zip 파일을 찾지 못했습니다. 수동으로 DATA_ROOT를 설정하세요.')
    DATA_ROOT = BISTOOL_ROOT or DRIVE_ROOT

# ============================================================
# 4단계: FaceScape 데이터 (FACESCAPE_ROOT) 탐색
# ============================================================
FACESCAPE_ROOT = None

facescape_candidates = []
if BISTOOL_ROOT:
    facescape_candidates.extend([
        os.path.join(BISTOOL_ROOT, 'FaceScape_Sqsh'),
        os.path.join(BISTOOL_ROOT, 'FaceScape'),
        os.path.join(BISTOOL_ROOT, 'Facescape'),
        os.path.join(BISTOOL_ROOT, 'Data', 'Facescape'),
        os.path.join(BISTOOL_ROOT, 'Data', 'FaceScape'),
        os.path.join(BISTOOL_ROOT, 'Data', 'FaceScape_Sqsh'),
    ])
facescape_candidates.extend([
    os.path.join(DRIVE_ROOT, 'FaceScape_Sqsh'),
    os.path.join(DRIVE_ROOT, 'Facescape'),
])

for candidate in facescape_candidates:
    if os.path.isdir(candidate):
        FACESCAPE_ROOT = candidate
        break

if FACESCAPE_ROOT:
    print(f'\n✅ FaceScape 데이터 루트: {FACESCAPE_ROOT}')
    print('    내용:')
    for item in sorted(os.listdir(FACESCAPE_ROOT)):
        full = os.path.join(FACESCAPE_ROOT, item)
        if os.path.isdir(full):
            print(f'      📁 {item}')
        else:
            size_gb = os.path.getsize(full) / (1024**3)
            print(f'      📄 {item} ({size_gb:.2f} GB)')
else:
    print(f'\n⚠️ FaceScape 폴더를 찾지 못했습니다.')
    print('    수동 설정 예: FACESCAPE_ROOT = "/content/drive/MyDrive/BISTOOL/FaceScape_Sqsh"')

print('\n=== 경로 탐색 완료 ===')

In [ ]:
# 작업 디렉토리 및 결과 저장 경로 설정
WORK_DIR = '/content/hrn_workspace'
RESULTS_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_Results')
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, 'HRN'), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, 'CodeFormer'), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, 'GFPGAN'), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, 'Comparison'), exist_ok=True)
print(f'작업 디렉토리: {WORK_DIR}')
print(f'결과 저장: {RESULTS_DIR}')

---
## 2. 의존성 설치

In [ ]:
# ============================================================
# 기본 패키지 설치 (Google Drive wheel 캐시 → 재시작 시 15~30초)
# ============================================================
import torch, sys, os

PACKAGES = [
    'pydicom', 'scipy', 'scikit-image', 'trimesh', 'matplotlib', 'plotly',
    'opencv-python-headless', 'pillow', 'tqdm', 'einops', 'kornia',
    'face-alignment', 'lpips', 'torchmetrics'
]
PKG_STR = ' '.join(PACKAGES)

cache_key = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
WHEEL_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'general_wheels', cache_key)
MARKER = os.path.join(WHEEL_DIR, '.complete')

if os.path.exists(MARKER):
    print(f'📦 캐시된 wheel에서 설치 중... ({cache_key})')
    ret = os.system(f'pip install -q --no-index --find-links "{WHEEL_DIR}" {PKG_STR}')
    if ret == 0:
        print('✅ 기본 패키지 캐시 설치 완료!')
    else:
        print('⚠️ 캐시 설치 실패, PyPI에서 재설치...')
        os.system(f'pip install -q {PKG_STR}')
else:
    print(f'⏳ 최초 설치 + wheel 캐시 생성 중... ({cache_key})')
    os.makedirs(WHEEL_DIR, exist_ok=True)
    !pip wheel -q -w "{WHEEL_DIR}" {PKG_STR}
    ret = os.system(f'pip install -q --no-index --find-links "{WHEEL_DIR}" {PKG_STR}')
    if ret == 0:
        with open(MARKER, 'w') as f:
            f.write(cache_key)
        print('✅ 기본 패키지 설치 + 캐시 저장 완료!')
    else:
        print('⚠️ wheel 설치 실패, PyPI에서 직접 설치...')
        os.system(f'pip install -q {PKG_STR}')

In [ ]:
# ============================================================
# HRN 의존성: ModelScope (Google Drive wheel 캐시)
# ============================================================
import time
_install_start = time.time()

PACKAGES_MS = ['modelscope', 'msdatasets', 'datasets']
PACKAGES_CV = ['addict', 'yapf', 'onnx']
ALL_PKGS = PACKAGES_MS + PACKAGES_CV
PKG_STR = ' '.join(ALL_PKGS)

cache_key = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
WHEEL_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'modelscope_wheels', cache_key)
MARKER = os.path.join(WHEEL_DIR, '.complete')

if os.path.exists(MARKER):
    print(f'\U0001f4e6 ModelScope 캐시에서 설치 중... ({cache_key})')
    ret = os.system(f'pip install -q --no-index --find-links "{WHEEL_DIR}" {PKG_STR} 2>/dev/null')
    if ret == 0:
        print('\u2705 ModelScope 캐시 설치 완료!')
    else:
        print('\u26a0\ufe0f 캐시 설치 실패, PyPI에서 재설치...')
        os.system(f'pip install -q {" ".join(PACKAGES_MS)}')
        os.system(f'pip install -q {" ".join(PACKAGES_CV)} 2>/dev/null || true')
else:
    print(f'\u23f3 ModelScope 최초 설치 + wheel 캐시 생성 중... ({cache_key})')
    os.makedirs(WHEEL_DIR, exist_ok=True)
    !pip wheel -q -w "{WHEEL_DIR}" {' '.join(PACKAGES_MS)}
    !pip wheel -q -w "{WHEEL_DIR}" {' '.join(PACKAGES_CV)} 2>/dev/null || true
    ret = os.system(f'pip install -q --no-index --find-links "{WHEEL_DIR}" {PKG_STR} 2>/dev/null')
    if ret == 0:
        with open(MARKER, 'w') as f:
            f.write(cache_key)
    else:
        os.system(f'pip install -q {" ".join(PACKAGES_MS)}')
        os.system(f'pip install -q {" ".join(PACKAGES_CV)} 2>/dev/null || true')
    print('\u2705 ModelScope 설치 완료!')

os.environ['PIP_CACHE_DIR'] = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache')
_install_elapsed = time.time() - _install_start
print(f'   설치 시간: {_install_elapsed:.0f}초')

In [ ]:
# ============================================================
# PyTorch3D 설치 (Google Drive 캐시 활용 → 재시작 시 < 1분)
# ★ wheel 파일명은 절대 변경하지 않음 (pip 호환성 태그 보존)
# ============================================================
import torch, sys, os, glob, shutil

pyt_version_str = torch.__version__.split('+')[0].replace('.', '')
version_str = ''.join([
    f'py3{sys.version_info.minor}',
    '_cu', torch.version.cuda.replace('.', ''),
    f'_pyt{pyt_version_str}'
])
print(f'PyTorch: {torch.__version__}, CUDA: {torch.version.cuda}, Python: 3.{sys.version_info.minor}')
print(f'Wheel key: {version_str}')

!pip install -q fvcore iopath

# ── 이미 설치되어 있으면 스킵 ──
try:
    import pytorch3d
    print(f'✅ PyTorch3D {pytorch3d.__version__} 이미 설치됨 — 스킵!')
except ImportError:
    # ── Google Drive 캐시 (버전별 하위 폴더) ──
    WHEEL_CACHE = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'pytorch3d_wheels', version_str)
    os.makedirs(WHEEL_CACHE, exist_ok=True)
    cached = sorted(glob.glob(os.path.join(WHEEL_CACHE, 'pytorch3d*.whl')))

    installed = False

    if cached:
        whl = cached[-1]
        print(f'📦 캐시된 wheel 발견: {os.path.basename(whl)}')
        ret = os.system(f'pip install -q "{whl}"')
        if ret == 0:
            try:
                import pytorch3d
                installed = True
                print('✅ PyTorch3D 캐시에서 설치 완료!')
            except ImportError:
                print('⚠️ 캐시 wheel 호환 안 됨 — 삭제 후 재빌드')
                shutil.rmtree(WHEEL_CACHE)
                os.makedirs(WHEEL_CACHE, exist_ok=True)
        else:
            print('⚠️ 캐시 wheel 설치 실패 — 삭제 후 재빌드')
            shutil.rmtree(WHEEL_CACHE)
            os.makedirs(WHEEL_CACHE, exist_ok=True)

    if not installed:
        print('⏳ PyTorch3D 설치 중... (wheel을 빌드하여 Drive에 캐시합니다)')
        WHEEL_TMP = '/tmp/pytorch3d_wheels'
        os.makedirs(WHEEL_TMP, exist_ok=True)

        # 1) Pre-built wheel 시도
        !pip download --no-deps --no-cache-dir -d {WHEEL_TMP} \
            pytorch3d \
            -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html \
            2>/dev/null

        wheels = glob.glob(f'{WHEEL_TMP}/pytorch3d*.whl')
        if not wheels:
            # 2) 소스에서 빌드
            print('Pre-built wheel 없음 → 소스에서 빌드합니다...')
            !pip wheel --no-build-isolation --no-deps \
                "git+https://github.com/facebookresearch/pytorch3d.git" \
                -w {WHEEL_TMP}
            wheels = glob.glob(f'{WHEEL_TMP}/pytorch3d*.whl')

        if wheels:
            whl = wheels[0]
            ret = os.system(f'pip install -q "{whl}"')
            if ret == 0:
                # Drive 캐시에 원본 파일명 그대로 저장
                dest = os.path.join(WHEEL_CACHE, os.path.basename(whl))
                shutil.copy2(whl, dest)
                print(f'💾 캐시 저장 완료: {version_str}/{os.path.basename(whl)}')
                print('✅ 다음 런타임 재시작 시 < 1분 내 설치됩니다!')
            else:
                print('❌ wheel 설치 실패')
        else:
            print('❌ wheel 빌드 실패')

    # 최종 확인
    try:
        import pytorch3d
        print(f'🔧 PyTorch3D {pytorch3d.__version__} 준비 완료')
    except ImportError:
        print('❌ PyTorch3D를 사용할 수 없습니다. 런타임 재시작 후 다시 시도하세요.')

In [ ]:
# ============================================================
# nvdiffrast 설치 (Google Drive 캐시 활용 → 재시작 시 빠른 설치)
# ★ CUDA_HOME, OpenGL 헤더, C++ 빌드 도구가 모두 필요합니다
# ★ wheel 파일명은 절대 변경하지 않음 (pip 호환성 태그 보존)
# ============================================================
import subprocess, sys, os, glob, shutil

# 1) CUDA 환경변수 설정
cuda_home = None
for p in ['/usr/local/cuda', '/usr/local/cuda-12', '/usr/local/cuda-11']:
    if os.path.isdir(p):
        cuda_home = p
        break

if cuda_home:
    os.environ['CUDA_HOME'] = cuda_home
    os.environ['PATH'] = f"{cuda_home}/bin:{os.environ.get('PATH', '')}"
    print(f'CUDA_HOME={cuda_home}')
    !nvcc --version 2>/dev/null | grep release || echo "nvcc not found in PATH"
else:
    print('⚠️ CUDA 디렉토리를 찾을 수 없습니다. GPU 런타임인지 확인하세요.')

# 2) 시스템 빌드 의존성 (Google Drive .deb 캐시)
import subprocess
_ubuntu_ver = subprocess.run(['lsb_release', '-rs'], capture_output=True, text=True).stdout.strip().replace('.', '')
DEB_CACHE = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'apt_debs', f'ubuntu{_ubuntu_ver}')
DEB_MARKER = os.path.join(DEB_CACHE, '.complete')
APT_PKGS = 'build-essential ninja-build freeglut3-dev libgl1-mesa-dev libgles2-mesa-dev libglew-dev libegl1-mesa-dev'

if os.path.exists(DEB_MARKER):
    print(f'📦 시스템 패키지 캐시에서 설치 중... (ubuntu{_ubuntu_ver})')
    !dpkg -i {DEB_CACHE}/*.deb > /dev/null 2>&1 || true
    !apt-get -f install -y -qq > /dev/null 2>&1
    print('✅ 시스템 패키지 캐시 설치 완료!')
else:
    print('⏳ 시스템 패키지 설치 + .deb 캐시 생성 중...')
    os.makedirs(DEB_CACHE, exist_ok=True)
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -y -qq --download-only {APT_PKGS} > /dev/null 2>&1
    !cp /var/cache/apt/archives/*.deb {DEB_CACHE}/ 2>/dev/null || true
    !apt-get install -y -qq {APT_PKGS} > /dev/null 2>&1
    with open(DEB_MARKER, 'w') as f:
        f.write(f'ubuntu{_ubuntu_ver}')
    print('✅ 시스템 패키지 설치 + 캐시 저장 완료!')

# 3) Python 빌드 도구
!pip install -q ninja cmake setuptools wheel

# 4) nvdiffrast 설치 (버전별 하위 폴더 캐시)
try:
    import nvdiffrast.torch
    print(f'\n✅ nvdiffrast 이미 설치되어 있습니다!')
except ImportError:
    # Python + CUDA 버전으로 캐시 키 생성
    cache_key = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
    WHEEL_CACHE = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'nvdiffrast_wheels', cache_key)
    os.makedirs(WHEEL_CACHE, exist_ok=True)
    cached = sorted(glob.glob(os.path.join(WHEEL_CACHE, 'nvdiffrast*.whl')))

    installed = False

    if cached:
        whl = cached[-1]
        print(f'📦 캐시된 wheel 발견: {os.path.basename(whl)}')
        ret = os.system(f'pip install -q "{whl}"')
        if ret == 0:
            try:
                import nvdiffrast.torch
                installed = True
                print('✅ nvdiffrast 캐시에서 설치 완료!')
            except ImportError:
                print('⚠️ 캐시 wheel 호환 안 됨 — 삭제 후 재빌드')
                shutil.rmtree(WHEEL_CACHE)
                os.makedirs(WHEEL_CACHE, exist_ok=True)
        else:
            print('⚠️ 캐시 wheel 설치 실패 — 삭제 후 재빌드')
            shutil.rmtree(WHEEL_CACHE)
            os.makedirs(WHEEL_CACHE, exist_ok=True)

    if not installed:
        print('⏳ nvdiffrast 최초 설치 중... (Drive에 캐시 저장)')
        WHEEL_TMP = '/tmp/nvdiffrast_wheels'
        os.makedirs(WHEEL_TMP, exist_ok=True)

        !pip wheel --no-build-isolation --no-deps \
            "git+https://github.com/NVlabs/nvdiffrast.git@v0.3.5" \
            -w {WHEEL_TMP}

        wheels = glob.glob(f'{WHEEL_TMP}/nvdiffrast*.whl')
        if wheels:
            whl = wheels[0]
            ret = os.system(f'pip install -q "{whl}"')
            if ret == 0:
                # Drive 캐시에 원본 파일명 그대로 저장
                dest = os.path.join(WHEEL_CACHE, os.path.basename(whl))
                shutil.copy2(whl, dest)
                print(f'\n💾 캐시 저장 완료: {cache_key}/{os.path.basename(whl)}')
            else:
                print('❌ wheel 설치 실패')
        else:
            # Fallback: 직접 설치
            print('wheel 빌드 실패 → 직접 설치 시도...')
            !rm -rf /tmp/nvdiffrast
            !git clone --quiet --branch v0.3.5 https://github.com/NVlabs/nvdiffrast.git /tmp/nvdiffrast
            !pip install --no-build-isolation -q /tmp/nvdiffrast

    # 최종 검증
    try:
        import nvdiffrast.torch
        print(f'\n✅ nvdiffrast 설치 완료!')
    except ImportError as e:
        print(f'\n❌ nvdiffrast 설치 실패: {e}')
        print('런타임 → GPU 확인 후 다시 실행하세요.')

---
## 3. 데이터 탐색 및 DICOM CT 시각화

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage import measure
import shutil
import zipfile
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# ★ 핵심: Drive zip → Colab 로컬로 복사 후 해제 (Drive 직접 읽기 100배 빠름)
# ============================================================
LOCAL_DATA = '/content/ct_data'
os.makedirs(LOCAL_DATA, exist_ok=True)

ct_folders = {}
for folder_name in ['002', '003', '004', '005', '007', '009']:
    local_folder = os.path.join(LOCAL_DATA, folder_name)
    zip_path = os.path.join(DATA_ROOT, f'{folder_name}.zip')

    # 이미 로컬에 해제됐으면 건너뜀
    if os.path.exists(local_folder) and len(glob.glob(os.path.join(local_folder, '**/*.dcm'), recursive=True)) > 0:
        dcm_count = len(glob.glob(os.path.join(local_folder, '**/*.dcm'), recursive=True))
        ct_folders[folder_name] = {'path': local_folder, 'count': dcm_count}
        print(f'  [{folder_name}] 이미 로컬에 있음 ({dcm_count} slices)')
        continue

    # zip 파일이 Drive에 있으면 로컬로 복사 후 해제
    if os.path.exists(zip_path):
        local_zip = os.path.join(LOCAL_DATA, f'{folder_name}.zip')
        print(f'  [{folder_name}] Drive → 로컬 복사 중...', end=' ', flush=True)
        shutil.copy2(zip_path, local_zip)
        print('해제 중...', end=' ', flush=True)
        with zipfile.ZipFile(local_zip, 'r') as zf:
            zf.extractall(LOCAL_DATA)
        os.remove(local_zip)  # zip 삭제 (로컬 용량 절약)

        # 해제된 폴더 확인
        if os.path.exists(local_folder):
            dcm_count = len(glob.glob(os.path.join(local_folder, '**/*.dcm'), recursive=True))
            ct_folders[folder_name] = {'path': local_folder, 'count': dcm_count}
            print(f'완료! ({dcm_count} slices)')
        else:
            print(f'해제됐지만 폴더 구조 확인 필요')
            # 하위 폴더 탐색
            for sub in os.listdir(LOCAL_DATA):
                sub_path = os.path.join(LOCAL_DATA, sub)
                if os.path.isdir(sub_path):
                    dcms = glob.glob(os.path.join(sub_path, '**/*.dcm'), recursive=True)
                    if dcms:
                        print(f'    → {sub}: {len(dcms)} dcm files')
    else:
        # zip 없으면 폴더 직접 확인 (Drive 내 해제된 폴더)
        drive_folder = os.path.join(DATA_ROOT, folder_name)
        if os.path.exists(drive_folder):
            print(f'  [{folder_name}] zip 없음, Drive 폴더를 로컬로 복사 중...')
            shutil.copytree(drive_folder, local_folder)
            dcm_count = len(glob.glob(os.path.join(local_folder, '**/*.dcm'), recursive=True))
            ct_folders[folder_name] = {'path': local_folder, 'count': dcm_count}
            print(f'    완료! ({dcm_count} slices)')

print(f'\n총 {len(ct_folders)}개의 CT 데이터셋 준비 완료 (로컬: {LOCAL_DATA})')

# DICOM 로드 함수 (로컬에서 읽으므로 빠름)
def load_dicom_series(folder_path):
    """DICOM 시리즈를 로드하여 3D 볼륨으로 반환합니다."""
    dcm_files = []
    for root, dirs, files in os.walk(folder_path):
        for f in files:
            if f.endswith('.dcm'):
                dcm_files.append(os.path.join(root, f))
    dcm_files.sort()

    slices = [pydicom.dcmread(f) for f in dcm_files]
    slices.sort(key=lambda s: float(getattr(s, 'ImagePositionPatient', [0,0,0])[2]))

    pixel_spacing = [float(x) for x in slices[0].PixelSpacing]
    slice_thickness = float(slices[0].SliceThickness)
    spacing = np.array([slice_thickness, pixel_spacing[0], pixel_spacing[1]])

    volume = np.stack([s.pixel_array.astype(np.float32) for s in slices])

    intercept = float(getattr(slices[0], 'RescaleIntercept', 0))
    slope = float(getattr(slices[0], 'RescaleSlope', 1))
    volume = volume * slope + intercept

    info = {
        'patient_id': str(getattr(slices[0], 'PatientID', 'Unknown')),
        'modality': str(getattr(slices[0], 'Modality', 'Unknown')),
        'rows': slices[0].Rows,
        'cols': slices[0].Columns,
        'num_slices': len(slices),
        'spacing': spacing,
    }
    return volume, spacing, info

In [ ]:
# 첫 번째 CT 데이터 로드 및 시각화
if not ct_folders:
    print('❌ CT 데이터가 없습니다. 위 셀을 확인하세요.')
else:
    first_folder = list(ct_folders.keys())[0]
    print(f'로딩 중: {first_folder}...')
    volume, spacing, info = load_dicom_series(ct_folders[first_folder]['path'])
    print(f'볼륨 shape: {volume.shape}')
    print(f'Spacing (mm): {spacing}')
    print(f'정보: {info}')

    # 3축 단면 시각화
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    mid_z = volume.shape[0] // 2
    mid_y = volume.shape[1] // 2
    mid_x = volume.shape[2] // 2

    axes[0].imshow(volume[mid_z], cmap='gray', vmin=-200, vmax=800)
    axes[0].set_title(f'Axial (Z={mid_z})')
    axes[0].axis('off')

    axes[1].imshow(volume[:, mid_y, :], cmap='gray', vmin=-200, vmax=800, aspect=spacing[0]/spacing[2])
    axes[1].set_title(f'Coronal (Y={mid_y})')
    axes[1].axis('off')

    axes[2].imshow(volume[:, :, mid_x], cmap='gray', vmin=-200, vmax=800, aspect=spacing[0]/spacing[1])
    axes[2].set_title(f'Sagittal (X={mid_x})')
    axes[2].axis('off')

    plt.suptitle(f'CT Scan - Patient {first_folder}', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'ct_overview_{first_folder}.png'), dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# CT 볼륨에서 얼굴 피부 표면 3D 추출 (Marching Cubes)
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def extract_face_surface(volume, spacing, threshold=200):
    """CT 볼륨에서 피부 표면을 Marching Cubes로 추출합니다.
    threshold: HU 값 (피부 표면 ~200-300 HU)
    """
    verts, faces, normals, values = measure.marching_cubes(
        volume, level=threshold, spacing=tuple(spacing)
    )
    return verts, faces, normals
if 'volume' not in dir() or volume is None:
    print('⚠️ CT 데이터가 로드되지 않았습니다. Cell 13을 먼저 실행하세요.')
else:

    print('피부 표면 추출 중 (Marching Cubes)...')
    verts, faces, normals = extract_face_surface(volume, spacing, threshold=200)
    print(f'Vertices: {verts.shape[0]:,}, Faces: {faces.shape[0]:,}')

    # 3D 표면 시각화 (Poly3DCollection 사용 - plot_trisurf보다 훨씬 빠름)
    print('3D 시각화 렌더링 중...')
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # 서브샘플링 (최대 20000개 면으로 제한 → 빠른 렌더링)
    max_faces = 20000
    step = max(1, len(faces) // max_faces)
    sampled_faces = faces[::step]

    # Poly3DCollection으로 렌더링 (plot_trisurf보다 10배+ 빠름)
    mesh_verts = verts[sampled_faces]  # (N_faces, 3, 3)
    # 축 순서: Z, Y, X → 시각화용 X, Y, Z로 변환
    mesh_verts_reorder = mesh_verts[:, :, [2, 1, 0]]

    poly = Poly3DCollection(mesh_verts_reorder, alpha=0.8, linewidths=0)
    poly.set_facecolor('peachpuff')
    ax.add_collection3d(poly)

    # 축 범위 설정
    ax.set_xlim(verts[:, 2].min(), verts[:, 2].max())
    ax.set_ylim(verts[:, 1].min(), verts[:, 1].max())
    ax.set_zlim(verts[:, 0].min(), verts[:, 0].max())

    ax.set_title(f'3D Face Surface - Patient {first_folder}\n({sampled_faces.shape[0]:,} faces displayed)')
    ax.view_init(elev=180, azim=270)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'ct_3d_surface_{first_folder}.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print('완료!')

In [ ]:
# CT 볼륨에서 2D 얼굴 이미지 렌더링 (정면, 측면)
def render_face_from_ct(volume, spacing, view='frontal'):
    """CT 볼륨에서 Maximum Intensity Projection으로 2D 얼굴 이미지를 생성합니다."""
    soft_tissue = np.clip(volume, -100, 400)
    soft_tissue = (soft_tissue - (-100)) / (400 - (-100)) * 255
    soft_tissue = soft_tissue.astype(np.uint8)

    if view == 'frontal':
        projection = np.max(soft_tissue[:, :soft_tissue.shape[1]//2, :], axis=1)
    elif view == 'lateral':
        projection = np.max(soft_tissue[:, :, :soft_tissue.shape[2]//2], axis=2)
    else:
        projection = np.max(soft_tissue, axis=0)

    return projection
if 'volume' not in dir() or volume is None:
    print('⚠️ CT 데이터가 로드되지 않았습니다. Cell 13을 먼저 실행하세요.')
else:

    # 정면 및 측면 렌더링
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    frontal = render_face_from_ct(volume, spacing, 'frontal')
    lateral = render_face_from_ct(volume, spacing, 'lateral')
    axial = render_face_from_ct(volume, spacing, 'axial')

    axes[0].imshow(frontal, cmap='gray')
    axes[0].set_title('Frontal MIP')
    axes[0].axis('off')

    axes[1].imshow(lateral, cmap='gray')
    axes[1].set_title('Lateral MIP')
    axes[1].axis('off')

    axes[2].imshow(axial, cmap='gray')
    axes[2].set_title('Axial MIP')
    axes[2].axis('off')

    plt.suptitle(f'CT-derived Face Projections - Patient {first_folder}', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'ct_projections_{first_folder}.png'), dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
import cv2
from PIL import Image

def extract_face_slice_image(volume, spacing):
    """CT 볼륨에서 얼굴 영역의 대표 축방향 슬라이스 이미지들을 추출합니다."""
    windowed = np.clip(volume, -100, 300)
    windowed = ((windowed + 100) / 400 * 255).astype(np.uint8)
    num_slices = volume.shape[0]
    face_start = int(num_slices * 0.35)
    face_end = int(num_slices * 0.75)
    face_slices = windowed[face_start:face_end]
    tissue_counts = [(i, np.sum(s > 50)) for i, s in enumerate(face_slices)]
    tissue_counts.sort(key=lambda x: x[1], reverse=True)
    best_indices = [face_start + tissue_counts[i][0] for i in range(min(5, len(tissue_counts)))]
    return windowed, best_indices

def extract_and_save_face_images(volume, spacing, patient_id, output_dir):
    """CT 볼륨에서 HRN 입력용 2D 얼굴 이미지를 추출하여 저장합니다."""
    os.makedirs(output_dir, exist_ok=True)
    saved_paths = []

    soft = np.clip(volume, -100, 400)
    soft = ((soft + 100) / 500 * 255).astype(np.uint8)

    half_y = soft.shape[1] // 2
    frontal_avg = np.mean(soft[:, :half_y, :], axis=1).astype(np.uint8)
    frontal_path = os.path.join(output_dir, f'{patient_id}_frontal.png')
    cv2.imwrite(frontal_path, frontal_avg)
    saved_paths.append(frontal_path)

    windowed, best_indices = extract_face_slice_image(volume, spacing)
    for i, idx in enumerate(best_indices[:3]):
        slice_path = os.path.join(output_dir, f'{patient_id}_axial_{i}.png')
        cv2.imwrite(slice_path, windowed[idx])
        saved_paths.append(slice_path)

    return saved_paths

# ★ 로컬에서 처리 → 결과만 Drive에 저장
LOCAL_FACES_DIR = '/content/extracted_faces'  # 로컬 (빠름)
FACE_IMAGES_DIR = os.path.join(RESULTS_DIR, 'extracted_faces')  # Drive (저장용)

all_face_images = {}

for folder_name, folder_info in ct_folders.items():
    print(f'처리 중: Patient {folder_name}...', end=' ', flush=True)
    vol, sp, inf = load_dicom_series(folder_info['path'])  # 로컬에서 읽으므로 빠름
    paths = extract_and_save_face_images(vol, sp, folder_name, LOCAL_FACES_DIR)
    all_face_images[folder_name] = paths
    print(f'완료! ({len(paths)} images, shape={vol.shape})')
    del vol

# 결과를 Drive에도 복사
os.makedirs(FACE_IMAGES_DIR, exist_ok=True)
shutil.copytree(LOCAL_FACES_DIR, FACE_IMAGES_DIR, dirs_exist_ok=True)

print(f'\n총 추출된 이미지 수: {sum(len(v) for v in all_face_images.values())}')
print(f'로컬 저장: {LOCAL_FACES_DIR}')
print(f'Drive 백업: {FACE_IMAGES_DIR}')

In [ ]:
# 추출된 얼굴 이미지 미리보기
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (patient_id, paths) in enumerate(all_face_images.items()):
    if i >= 6:
        break
    img = cv2.imread(paths[0], cv2.IMREAD_GRAYSCALE)
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f'Patient {patient_id}')
    axes[i].axis('off')

plt.suptitle('CT에서 추출된 얼굴 이미지 (정면 투영)', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'extracted_faces_overview.png'), dpi=150, bbox_inches='tight')
plt.show()

---
## 4. HRN 모델 - 3D 얼굴 복원

논문의 핵심 아키텍처:
- **Low-frequency**: 3DMM (BFM) 기반 coarse mesh 생성
- **Mid-frequency**: Deformation map (64x64, pix2pix) → 윤곽 디테일
- **High-frequency**: Displacement map (256x256, pix2pix) → 주름, 모공 등
- **De-retouching Module**: 피부 텍스처와 기하학적 디테일 분리

ModelScope의 사전학습 HRN 모델을 사용합니다.

In [ ]:
# ============================================================
# HRN 모델 초기화 (ModelScope 파이프라인)
# ★ 모델 가중치(~1GB)를 Google Drive에 캐시 → 다음 세션에서 재다운로드 불필요
# ============================================================
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

# 모델 캐시를 Google Drive에 저장
MODEL_CACHE_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_Models', 'modelscope_cache')
os.makedirs(MODEL_CACHE_DIR, exist_ok=True)
os.environ['MODELSCOPE_CACHE'] = MODEL_CACHE_DIR

# 캐시 존재 여부 확인
HRN_MODEL_ID = 'damo/cv_resnet50_face-reconstruction'
cached_model_dir = os.path.join(MODEL_CACHE_DIR, 'hub', HRN_MODEL_ID.replace('/', os.sep))
if os.path.exists(cached_model_dir):
    print(f'✅ HRN 모델이 Drive에 캐시되어 있습니다: {cached_model_dir}')
    print('   다운로드 없이 바로 로딩합니다.')
else:
    print('⬇️ HRN 모델 최초 다운로드 중 (~1GB, Drive에 캐시됩니다)...')

import time
_load_start = time.time()

# v2.0.3 (최신) 시도 → 실패 시 v2.0.0-HRN (원본)
for _rev in ['v2.0.3', 'v2.0.0-HRN']:
    try:
        face_reconstruction = pipeline(
            Tasks.face_reconstruction,
            model=HRN_MODEL_ID,
            model_revision=_rev
        )
        print(f'✅ HRN 모델 로딩 완료! (revision: {_rev}, {time.time()-_load_start:.1f}초)')
        break
    except Exception as e:
        print(f'⚠️ revision {_rev} 실패: {e}')
else:
    raise RuntimeError('HRN 모델 로딩 실패 — 모든 revision 시도 완료')

print(f'   캐시 위치: {MODEL_CACHE_DIR}')


In [ ]:
from modelscope.outputs import OutputKeys
import trimesh
import json
import time

def run_hrn_reconstruction(pipeline_model, image_path, output_dir, patient_id):
    """HRN 모델로 단일 이미지에서 3D 얼굴 복원을 수행합니다."""
    os.makedirs(output_dir, exist_ok=True)

    start_time = time.time()
    result = pipeline_model(image_path)
    elapsed = time.time() - start_time

    output_info = {'patient_id': patient_id, 'input_image': image_path, 'inference_time': elapsed}
    saved_files = {}

    # 3D Mesh (OBJ) 저장
    if OutputKeys.OUTPUT in result:
        mesh_data = result[OutputKeys.OUTPUT]
        if 'mesh' in mesh_data:
            obj_path = os.path.join(output_dir, f'{patient_id}_hrn_mesh.obj')
            # write_obj 유틸리티 사용
            try:
                from modelscope.models.cv.face_reconstruction.utils import write_obj
                write_obj(obj_path, mesh_data['mesh'])
                saved_files['mesh'] = obj_path
            except Exception:
                # 수동 OBJ 저장
                if hasattr(mesh_data['mesh'], 'vertices'):
                    mesh_obj = trimesh.Trimesh(
                        vertices=mesh_data['mesh'].vertices,
                        faces=mesh_data['mesh'].faces
                    )
                    mesh_obj.export(obj_path)
                    saved_files['mesh'] = obj_path

    # 시각화 이미지 저장 (vis_image from OUTPUT dict)
    output_dict = result.get(OutputKeys.OUTPUT, {})
    vis_img = output_dict.get('vis_image') if isinstance(output_dict, dict) else None

    # fallback: OUTPUT_IMG (텍스처맵) 또는 입력 이미지
    if vis_img is None and OutputKeys.OUTPUT_IMG in result:
        vis_img = result[OutputKeys.OUTPUT_IMG]
    if vis_img is None:
        vis_img = cv2.imread(image_path)

    if vis_img is not None:
        vis_path = os.path.join(output_dir, f'{patient_id}_hrn_visualization.jpg')
        if isinstance(vis_img, np.ndarray):
            cv2.imwrite(vis_path, vis_img)
        else:
            with open(vis_path, 'wb') as f:
                f.write(vis_img)
        saved_files['visualization'] = vis_path

    # 텍스처 저장
    if 'texture' in result.get(OutputKeys.OUTPUT, {}):
        tex_path = os.path.join(output_dir, f'{patient_id}_hrn_texture.png')
        tex = result[OutputKeys.OUTPUT]['texture']
        if isinstance(tex, np.ndarray):
            cv2.imwrite(tex_path, tex)
        saved_files['texture'] = tex_path

    output_info['saved_files'] = saved_files
    return result, output_info

print('HRN 추론 함수 정의 완료')


In [ ]:
# ============================================================
# ★ HRN 파이프라인 중간 결과 추출 (Intermediate Results Extraction)
# ============================================================
# 3DMM Coarse → Mid-Freq Deformation → High-Freq Displacement
# 각 단계의 알베도맵, 텍스처맵, 노멀맵, 변형맵, 변위맵, 메쉬를 캡처합니다
# ============================================================
import torch
from collections import OrderedDict
from modelscope.outputs import OutputKeys

if not all_face_images:
    print('❌ 추출된 얼굴 이미지가 없습니다. CT 데이터 셀을 먼저 실행하세요.')
    intermediates = OrderedDict()
    test_path = None
else:
    test_path = list(all_face_images.values())[0][0]
    print(f'입력 이미지: {test_path}\n')

    # ━━━ STEP 1: 파이프라인 출력 구조 전체 탐색 ━━━
    raw_result = face_reconstruction(test_path)

    def inspect_obj(obj, name='root', depth=0, max_depth=3):
        indent = '  ' * depth
        if depth > max_depth:
            return
        if isinstance(obj, np.ndarray):
            print(f'{indent}ndarray  {name}: {obj.shape} [{obj.min():.2f}~{obj.max():.2f}]')
        elif isinstance(obj, torch.Tensor):
            print(f'{indent}Tensor   {name}: {tuple(obj.shape)}')
        elif isinstance(obj, dict):
            print(f'{indent}dict     {name}: ({len(obj)} keys)')
            for k, v in obj.items():
                inspect_obj(v, str(k), depth+1, max_depth)
        elif isinstance(obj, (list, tuple)):
            print(f'{indent}{type(obj).__name__:8s} {name}: (len={len(obj)})')
            for i, v in enumerate(obj[:3]):
                inspect_obj(v, f'[{i}]', depth+1, max_depth)
        elif hasattr(obj, 'vertices'):
            print(f'{indent}Mesh     {name}: V={obj.vertices.shape} F={obj.faces.shape}')
        else:
            print(f'{indent}{type(obj).__name__:8s} {name}')

    print('━' * 60)
    print('  STEP 1: 파이프라인 출력 구조')
    print('━' * 60)
    inspect_obj(raw_result)

    # ━━━ STEP 2: 모델 내부 서브모듈 구조 ━━━
    print('\n' + '━' * 60)
    print('  STEP 2: 모델 서브모듈 구조')
    print('━' * 60)

    hrn_model = face_reconstruction.model
    print(f'Pipeline model: {type(hrn_model).__name__}')

    actual_model = hrn_model
    for attr in ['model', 'net', 'network', 'backbone', 'face_model']:
        if hasattr(actual_model, attr):
            inner = getattr(actual_model, attr)
            if hasattr(inner, 'named_modules') or hasattr(inner, 'forward'):
                print(f'  └→ .{attr} = {type(inner).__name__}')
                actual_model = inner

    if hasattr(actual_model, 'named_children'):
        print(f'\n  서브모듈:')
        for name, mod in actual_model.named_children():
            n_p = sum(p.numel() for p in mod.parameters()) / 1e6
            extra = f' ({n_p:.1f}M params)' if n_p > 0.01 else ''
            print(f'    {name}: {type(mod).__name__}{extra}')

    # ━━━ STEP 3: Forward Hook으로 중간 텐서 캡처 ━━━
    print('\n' + '━' * 60)
    print('  STEP 3: Forward Hook 캡처')
    print('━' * 60)

    intermediates = OrderedDict()

    def make_hook(name):
        def hook_fn(module, inp, out):
            try:
                if isinstance(out, torch.Tensor) and out.dim() >= 2:
                    intermediates[name] = out.detach().cpu().numpy()
                elif isinstance(out, dict):
                    for k, v in out.items():
                        if isinstance(v, torch.Tensor) and v.dim() >= 2:
                            intermediates[f'{name}/{k}'] = v.detach().cpu().numpy()
                        elif isinstance(v, np.ndarray) and v.ndim >= 2:
                            intermediates[f'{name}/{k}'] = v
                elif isinstance(out, (tuple, list)):
                    for i, v in enumerate(out[:5]):
                        if isinstance(v, torch.Tensor) and v.dim() >= 2:
                            intermediates[f'{name}[{i}]'] = v.detach().cpu().numpy()
            except Exception:
                pass
        return hook_fn

    hook_kw = [
        '3dmm', 'bfm', 'morphable', 'coarse',
        'pix2pix', 'generator', 'unet', 'netG',
        'mid', 'deform', 'high', 'displace', 'detail',
        'albedo', 'texture', 'normal',
        'render', 'recon', 'decode', 'refine',
        'retouching', 'de_retouching',
    ]

    hook_handles = []
    target = actual_model if hasattr(actual_model, 'named_modules') else hrn_model

    if hasattr(target, 'named_modules'):
        for name, mod in target.named_modules():
            if any(kw in name.lower() for kw in hook_kw):
                hook_handles.append(mod.register_forward_hook(make_hook(name)))
                print(f'  Hook: {name} ({type(mod).__name__})')

    if not hook_handles and hasattr(target, 'named_children'):
        print('  (키워드 매칭 없음 → 최상위 서브모듈 전체 Hook)')
        for name, mod in target.named_children():
            hook_handles.append(mod.register_forward_hook(make_hook(name)))
            print(f'  Hook: {name} ({type(mod).__name__})')

    print(f'\n  Hook {len(hook_handles)}개 등록, 추론 실행 중...')
    hooked_result = face_reconstruction(test_path)

    for h in hook_handles:
        h.remove()

    # ━━━ STEP 4: OUTPUT dict에서 직접 추출 ━━━
    output_data = hooked_result.get(OutputKeys.OUTPUT, {})
    if isinstance(output_data, dict):
        for k, v in output_data.items():
            if isinstance(v, np.ndarray):
                intermediates[f'OUTPUT/{k}'] = v
            elif isinstance(v, torch.Tensor):
                intermediates[f'OUTPUT/{k}'] = v.detach().cpu().numpy()
            elif hasattr(v, 'vertices'):
                intermediates[f'OUTPUT/{k}'] = v

    if OutputKeys.OUTPUT_IMG in hooked_result:
        vis = hooked_result[OutputKeys.OUTPUT_IMG]
        if isinstance(vis, np.ndarray):
            intermediates['OUTPUT_IMG'] = vis

    # 결과 요약
    print(f'\n{"━" * 60}')
    print(f'  ✅ 총 {len(intermediates)}개 중간 결과 캡처 완료')
    print(f'{"━" * 60}')
    for k, v in intermediates.items():
        if isinstance(v, np.ndarray):
            print(f'  {k}: shape={v.shape}')
        elif hasattr(v, 'vertices'):
            print(f'  {k}: Mesh V={v.vertices.shape} F={v.faces.shape}')

    print(f'\n→ 다음 셀에서 시각화합니다.')

In [ ]:
# ============================================================
# ★ HRN 중간 결과 시각화 (알베도, 텍스처, 노멀, 변형/변위맵, 메쉬)
# ============================================================
from collections import OrderedDict
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

if not intermediates:
    print('⚠️ 중간 결과가 없습니다. 위 셀을 먼저 실행하세요.')
else:
    # ---- 2D 맵 시각화 ----
    image_maps = OrderedDict()

    if test_path:
        inp_img = cv2.imread(test_path)
        if inp_img is not None:
            image_maps['0_INPUT'] = cv2.cvtColor(inp_img, cv2.COLOR_BGR2RGB)

    for k, v in intermediates.items():
        if not isinstance(v, np.ndarray):
            continue
        img = None
        if v.ndim == 2 and min(v.shape) > 8:
            img = v
        elif v.ndim == 3 and v.shape[0] in [1, 3] and v.shape[1] > 8:
            img = np.transpose(v, (1, 2, 0)).squeeze()
        elif v.ndim == 3 and v.shape[-1] in [1, 3] and v.shape[0] > 8:
            img = v.squeeze()
        elif v.ndim == 4 and v.shape[0] == 1:
            v0 = v[0]
            if v0.shape[0] in [1, 3] and v0.shape[1] > 8:
                img = np.transpose(v0, (1, 2, 0)).squeeze()
            elif v0.shape[-1] in [1, 3] and v0.shape[0] > 8:
                img = v0.squeeze()
            elif v0.shape[0] > 3 and v0.shape[1] > 8:
                img = np.transpose(v0[:3], (1, 2, 0))
        if img is not None:
            image_maps[k] = img

    if 'OUTPUT_IMG' in intermediates:
        vis = intermediates['OUTPUT_IMG']
        if vis.ndim == 3 and vis.shape[-1] == 3:
            image_maps['Z_FINAL_VIS'] = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)

    n = len(image_maps)
    if n > 0:
        ncols = min(4, n)
        nrows = (n + ncols - 1) // ncols
        fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4.5 * nrows))
        if n == 1:
            axes = np.array([[axes]])
        else:
            axes = np.atleast_2d(axes)

        for idx, (name, img) in enumerate(image_maps.items()):
            r, c = idx // ncols, idx % ncols
            ax = axes[r, c]
            img_f = img.astype(np.float64)
            if img_f.max() > 1.5:
                img_f = np.clip(img_f, 0, 255) / 255.0
            elif img_f.min() < -0.5:
                img_f = (img_f - img_f.min()) / max(img_f.max() - img_f.min(), 1e-8)
            else:
                img_f = np.clip(img_f, 0, 1)

            if img_f.ndim == 2:
                im = ax.imshow(img_f, cmap='viridis')
                plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            else:
                ax.imshow(img_f)
            ax.set_title(name.replace('/', '\n'), fontsize=8, fontweight='bold')
            ax.axis('off')

        for idx in range(n, nrows * ncols):
            axes[idx // ncols, idx % ncols].set_visible(False)

        plt.suptitle('HRN Pipeline - 단계별 중간 결과', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, 'HRN', 'hrn_intermediate_maps.png'), dpi=200, bbox_inches='tight')
        plt.show()
        print(f'2D 맵 {n}개 시각화 완료')
    else:
        print('⚠️ 시각화 가능한 2D 맵 없음. 위 셀의 텐서 shape를 확인하세요.')

    # ---- 3D 메쉬 시각화 ----
    mesh_items = {k: v for k, v in intermediates.items() if hasattr(v, 'vertices')}
    if mesh_items:
        n_m = min(len(mesh_items), 3)
        fig_m = plt.figure(figsize=(6 * n_m, 5))
        for idx, (name, mesh_obj) in enumerate(list(mesh_items.items())[:3]):
            ax = fig_m.add_subplot(1, n_m, idx + 1, projection='3d')
            vm, fm = mesh_obj.vertices, mesh_obj.faces
            step = max(1, len(fm) // 15000)
            tris = vm[fm[::step]]
            poly = Poly3DCollection(tris, alpha=0.7, linewidths=0)
            poly.set_facecolor('peachpuff')
            ax.add_collection3d(poly)
            ax.set_xlim(vm[:, 0].min(), vm[:, 0].max())
            ax.set_ylim(vm[:, 1].min(), vm[:, 1].max())
            ax.set_zlim(vm[:, 2].min(), vm[:, 2].max())
            ax.set_title(f'{name}\nV={vm.shape[0]:,} F={fm.shape[0]:,}', fontsize=9)
        plt.suptitle('HRN - 3D Mesh (중간/최종)', fontsize=13)
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, 'HRN', 'hrn_intermediate_meshes.png'), dpi=150)
        plt.show()
        print(f'3D 메쉬 {len(mesh_items)}개 시각화 완료')

    print(f'\n저장 위치: {os.path.join(RESULTS_DIR, "HRN")}/')

In [ ]:
# ============================================================
# CT에서 추출한 얼굴 이미지로 HRN 3D 복원 수행
# ============================================================
HRN_OUTPUT_DIR = os.path.join(RESULTS_DIR, 'HRN')
hrn_results = {}

for patient_id, image_paths in all_face_images.items():
    # 정면 이미지 사용
    frontal_path = image_paths[0]
    print(f'\n=== Patient {patient_id} HRN 추론 중... ===')
    print(f'  입력: {frontal_path}')

    try:
        result, info = run_hrn_reconstruction(
            face_reconstruction, frontal_path,
            os.path.join(HRN_OUTPUT_DIR, patient_id), patient_id
        )
        hrn_results[patient_id] = info
        print(f'  추론 시간: {info["inference_time"]:.2f}초')
        print(f'  저장된 파일: {list(info["saved_files"].keys())}')
    except Exception as e:
        print(f'  오류 발생: {e}')
        hrn_results[patient_id] = {'error': str(e)}

# 결과 요약 저장
summary_path = os.path.join(HRN_OUTPUT_DIR, 'hrn_results_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(hrn_results, f, indent=2, default=str, ensure_ascii=False)
print(f'\n결과 요약 저장: {summary_path}')

In [ ]:
# HRN 결과 시각화
successful_results = {k: v for k, v in hrn_results.items() if 'error' not in v}

if successful_results:
    n_results = len(successful_results)
    fig, axes = plt.subplots(n_results, 3, figsize=(18, 6 * n_results))
    if n_results == 1:
        axes = axes[np.newaxis, :]

    for i, (patient_id, info) in enumerate(successful_results.items()):
        # 원본 입력 이미지
        input_img = cv2.imread(info['input_image'])
        if input_img is not None:
            axes[i, 0].imshow(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB))
        axes[i, 0].set_title(f'Input - Patient {patient_id}')
        axes[i, 0].axis('off')

        # HRN 시각화 결과
        vis_path = info['saved_files'].get('visualization')
        if vis_path and os.path.exists(vis_path):
            vis_img = cv2.imread(vis_path)
            if vis_img is not None:
                axes[i, 1].imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
        axes[i, 1].set_title(f'HRN Reconstruction')
        axes[i, 1].axis('off')

        # 텍스처 맵
        tex_path = info['saved_files'].get('texture')
        if tex_path and os.path.exists(tex_path):
            tex_img = cv2.imread(tex_path)
            if tex_img is not None:
                axes[i, 2].imshow(cv2.cvtColor(tex_img, cv2.COLOR_BGR2RGB))
        axes[i, 2].set_title(f'Texture Map')
        axes[i, 2].axis('off')

    plt.suptitle('HRN 3D Face Reconstruction Results', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'HRN', 'hrn_all_results.png'), dpi=150, bbox_inches='tight')
    plt.show()
else:
    print('성공적인 HRN 결과가 없습니다. 위의 오류를 확인하세요.')

---
## 5. FaceScape 데이터 실험

FaceScape 데이터셋으로 HRN 성능을 평가합니다.
- 3D 스캔 ground truth와 HRN 복원 결과 비교
- Chamfer Distance, Mean Normal Error 측정

In [ ]:
# ============================================================
# FaceScape 데이터 준비
# ★ .sqsh (SquashFS) 파일 → 마운트만 하면 즉시 읽기 가능 (해제 불필요!)
# ★ zip과 달리 디스크 공간 추가 사용 없음, 접근 속도도 빠름
# ============================================================
print(f'FaceScape 경로: {FACESCAPE_ROOT}')
LOCAL_FACESCAPE = '/content/facescape_data'


if FACESCAPE_ROOT is None or not os.path.exists(FACESCAPE_ROOT):
    print('❌ FaceScape 폴더를 찾을 수 없습니다.')
    print('   FACESCAPE_ROOT = "/content/drive/MyDrive/BISTOOL/FaceScape_Sqsh"')
else:
    # squashfs-tools 설치 (마운트에 필요)
    !apt-get install -y -qq squashfs-tools > /dev/null 2>&1

    # Drive 내 FaceScape 파일 목록
    print(f'\n📁 FaceScape 디렉토리 내용:')
    facescape_files = sorted(os.listdir(FACESCAPE_ROOT))
    sqsh_files = []
    zip_files = []
    dir_list = []

    for f in facescape_files:
        full = os.path.join(FACESCAPE_ROOT, f)
        if os.path.isdir(full):
            dir_list.append(f)
            print(f'  📁 {f}/')
        else:
            size_gb = os.path.getsize(full) / (1024**3)
            print(f'  📄 {f} ({size_gb:.2f} GB)')
            if f.endswith('.sqsh'):
                sqsh_files.append(f)
            elif f.endswith('.zip'):
                zip_files.append(f)

    # ============================================================
    # SquashFS 마운트 (해제 불필요, 디스크 추가 사용 없음)
    # ============================================================
    os.makedirs(LOCAL_FACESCAPE, exist_ok=True)
    mounted_paths = []

    if sqsh_files:
        print(f'\n✅ .sqsh 파일 {len(sqsh_files)}개 감지 → 마운트 방식 사용')

        for sqsh_name in sqsh_files:
            sqsh_path = os.path.join(FACESCAPE_ROOT, sqsh_name)
            mount_name = sqsh_name.replace('.sqsh', '')
            mount_point = os.path.join(LOCAL_FACESCAPE, mount_name)

            # 이미 마운트됐으면 건너뜀
            if os.path.ismount(mount_point):
                print(f'  [{mount_name}] 이미 마운트됨: {mount_point}')
                mounted_paths.append(mount_point)
                continue

            os.makedirs(mount_point, exist_ok=True)

            # ★ Drive에서 Colab 로컬로 복사 후 마운트 (Drive에서 직접 마운트하면 느림)
            local_sqsh = f'/content/{sqsh_name}'
            if not os.path.exists(local_sqsh):
                print(f'  [{mount_name}] Drive → Colab 복사 중...', end=' ', flush=True)
                import shutil
                shutil.copy2(sqsh_path, local_sqsh)
                print('완료!')

            print(f'  [{mount_name}] 마운트 중...', end=' ', flush=True)
            ret = os.system(f'sudo mount -t squashfs -o loop,ro "{local_sqsh}" "{mount_point}" 2>/dev/null')
            if ret == 0:
                mounted_paths.append(mount_point)
                # 내용 확인
                try:
                    items = os.listdir(mount_point)
                    print(f'완료! ({len(items)}개 항목)')
                    for item in items[:5]:
                        print(f'    {item}')
                    if len(items) > 5:
                        print(f'    ... 외 {len(items)-5}개')
                except Exception as e:
                    print(f'마운트됨 (내용 확인 실패: {e})')
            else:
                # 마운트 실패 시 unsquashfs로 일부만 해제
                print(f'마운트 실패, unsquashfs로 해제 시도...')
                !unsquashfs -f -d "{mount_point}" "{local_sqsh}" -e 5 2>/dev/null || \
                    unsquashfs -f -d "{mount_point}" "{local_sqsh}" 2>/dev/null
                if os.listdir(mount_point):
                    mounted_paths.append(mount_point)
                    print(f'  해제 완료!')

    elif zip_files:
        # sqsh가 없으면 zip fallback
        print(f'\n⚠️ .sqsh 없음, .zip fallback ({len(zip_files)}개)')
        import zipfile
        for z in zip_files[:1]:
            zip_path = os.path.join(FACESCAPE_ROOT, z)
            local_zip = f'/content/{z}'
            print(f'  [{z}] Drive → Colab 복사 중...')
            shutil.copy2(zip_path, local_zip)
            print(f'  Subject 1~5만 해제 중...')
            with zipfile.ZipFile(local_zip, 'r') as zf:
                for name in zf.namelist():
                    parts = name.split('/')
                    if len(parts) > 1 and parts[0].isdigit() and int(parts[0]) <= 5:
                        try:
                            zf.extract(name, LOCAL_FACESCAPE)
                        except Exception:
                            pass
            os.remove(local_zip)
            mounted_paths.append(LOCAL_FACESCAPE)
            print(f'  완료!')

    elif dir_list:
        print(f'\n폴더 감지 → 로컬 복사 중...')
        for d in dir_list[:1]:
            src = os.path.join(FACESCAPE_ROOT, d)
            dst = os.path.join(LOCAL_FACESCAPE, d)
            if not os.path.exists(dst):
                shutil.copytree(src, dst)
            mounted_paths.append(dst)
            print(f'  {d} → 복사 완료')

    # 이미지 파일 탐색
    local_images = []
    for mp in mounted_paths:
        for root, dirs, files in os.walk(mp):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    local_images.append(os.path.join(root, f))
            if len(local_images) > 100:
                break

    print(f'\n{"="*50}')
    print(f'FaceScape 이미지: {len(local_images)}개 발견')
    if local_images:
        print(f'  예: {local_images[0]}')
    print(f'마운트 경로: {mounted_paths}')

In [ ]:
# FaceScape 이미지로 HRN 복원 및 평가 (★ 로컬에서 처리)
facescape_hrn_results = {}  # 항상 초기화 (이후 셀에서 참조)

facescape_images = []
for root, dirs, files in os.walk(LOCAL_FACESCAPE):
    for f in files:
        if f.lower().endswith(('.jpg', '.png', '.jpeg')):
            facescape_images.append(os.path.join(root, f))

print(f'FaceScape 이미지 {len(facescape_images)}개 발견 (로컬: {LOCAL_FACESCAPE})')

if len(facescape_images) == 0:
    print('⚠️ FaceScape 이미지가 없습니다. FaceScape 데이터 준비 셀을 먼저 실행하세요.')
else:
    # 최대 10개 이미지로 HRN 테스트
    FACESCAPE_HRN_DIR = os.path.join(RESULTS_DIR, 'HRN', 'facescape')
    os.makedirs(FACESCAPE_HRN_DIR, exist_ok=True)

    test_images = facescape_images[:10]

    for i, img_path in enumerate(test_images):
        subject_id = f'facescape_{i:03d}'
        print(f'[{i+1}/{len(test_images)}] {os.path.basename(img_path)}...')

        try:
            result, info = run_hrn_reconstruction(
                face_reconstruction, img_path,
                os.path.join(FACESCAPE_HRN_DIR, subject_id), subject_id
            )
            facescape_hrn_results[subject_id] = info
            print(f'  완료 ({info["inference_time"]:.2f}s)')
        except Exception as e:
            print(f'  오류: {e}')
            facescape_hrn_results[subject_id] = {'error': str(e)}

    print(f'\nFaceScape HRN 결과: {len([v for v in facescape_hrn_results.values() if "error" not in v])} 성공')

---
## 6. 최신 기법 비교 - CodeFormer & GFPGAN

### 비교 대상 기법:
1. **CodeFormer** (NeurIPS 2022): Codebook 기반 Transformer로 얼굴 복원. 열화된 얼굴을 고품질로 복원.
2. **GFPGAN v1.4** (CVPR 2021): GAN 기반 얼굴 복원. GFP (Generative Facial Prior) 활용.

이 방법들은 2D 얼굴 복원(restoration) 기법으로, HRN의 3D 복원과 다른 관점에서 비교합니다:
- **HRN**: 2D → 3D 기하학적 복원 (mesh, texture)
- **CodeFormer/GFPGAN**: 2D → 2D 품질 복원 (해상도, 디테일 향상)

In [ ]:
# ============================================================
# 6-1. CodeFormer 설치 및 설정 (Google Drive 캐시)
# \u2605 basicsr는 site-packages에 설치하지 않음 (GFPGAN과 충돌 방지)
# \u2605 추론 시 cd CodeFormer로 실행 → 번들 basicsr 자동 사용
# ============================================================
CODEFORMER_DIR = os.path.join(WORK_DIR, 'CodeFormer')

# Drive 캐시 경로
CF_TAR = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'codeformer_cache', 'codeformer_full.tar.gz')
CF_TAR_MARKER = CF_TAR + '.complete'
cache_key = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
CF_WHEEL_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'codeformer_wheels', cache_key)
CF_WHEEL_MARKER = os.path.join(CF_WHEEL_DIR, '.complete')

if os.path.exists(CODEFORMER_DIR) and os.path.exists(os.path.join(CODEFORMER_DIR, 'inference_codeformer.py')):
    print(f'CodeFormer 이미 설치됨: {CODEFORMER_DIR}')
elif os.path.exists(CF_TAR) and os.path.exists(CF_TAR_MARKER):
    # Drive 캐시에서 복원 (30~60초)
    print('CodeFormer 캐시에서 복원 중...')
    !tar xzf "{CF_TAR}" -C {os.path.dirname(CODEFORMER_DIR)}
    # pip 의존성 복원 (basicsr 제외 - GFPGAN이 PyPI에서 설치)
    if os.path.exists(CF_WHEEL_MARKER):
        !pip install -q --no-index --find-links "{CF_WHEEL_DIR}" \
            lpips opencv-python scikit-image scipy pillow pyyaml tqdm addict 2>/dev/null || true
    else:
        !pip install -q lpips opencv-python scikit-image scipy pillow pyyaml tqdm addict 2>/dev/null || true
    print('\u2705 CodeFormer 캐시 복원 완료!')
else:
    # 최초 설치 + Drive에 캐시 저장
    print('CodeFormer 최초 설치 중...')
    !git clone https://github.com/sczhou/CodeFormer.git {CODEFORMER_DIR}

    # pip deps wheel 캐시 (basicsr, tb-nightly 제외)
    CF_DEPS = 'lpips opencv-python scikit-image scipy pillow pyyaml tqdm addict'
    os.makedirs(CF_WHEEL_DIR, exist_ok=True)
    !pip wheel -q -w "{CF_WHEEL_DIR}" {CF_DEPS} 2>/dev/null || true
    !pip install -q {CF_DEPS} 2>/dev/null || true
    with open(CF_WHEEL_MARKER, 'w') as f:
        f.write('ok')

    # pretrained 모델 다운로드
    !cd {CODEFORMER_DIR} && python scripts/download_pretrained_models.py facelib
    !cd {CODEFORMER_DIR} && python scripts/download_pretrained_models.py CodeFormer

    # tarball로 Drive에 저장
    os.makedirs(os.path.dirname(CF_TAR), exist_ok=True)
    !tar czf "{CF_TAR}" -C {os.path.dirname(CODEFORMER_DIR)} CodeFormer
    with open(CF_TAR_MARKER, 'w') as f:
        f.write('ok')
    print('\u2705 CodeFormer 설치 + Drive 캐시 저장 완료!')

print(f'CodeFormer 준비 완료: {CODEFORMER_DIR}')

In [ ]:
# ============================================================
# 6-2. GFPGAN 설치 및 설정 (Google Drive 캐시)
# \u2605 basicsr, facexlib, realesrgan를 먼저 설치 후 gfpgan 설치
# ============================================================
cache_key = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
GFPGAN_WHEEL_DIR = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache', 'gfpgan_wheels', cache_key)
GFPGAN_WHEEL_MARKER = os.path.join(GFPGAN_WHEEL_DIR, '.complete')

# 1) pip install (wheel 캐시)
GFPGAN_PKGS = 'basicsr facexlib gfpgan realesrgan'
if os.path.exists(GFPGAN_WHEEL_MARKER):
    print(f'GFPGAN 캐시에서 설치 중... ({cache_key})')
    ret = os.system(f'pip install -q --no-index --find-links "{GFPGAN_WHEEL_DIR}" {GFPGAN_PKGS}')
    if ret != 0:
        print('\u26a0\ufe0f 캐시 실패, PyPI에서 재설치...')
        os.system(f'pip install -q {GFPGAN_PKGS}')
else:
    print('GFPGAN 최초 설치 중...')
    os.makedirs(GFPGAN_WHEEL_DIR, exist_ok=True)
    !pip wheel -q -w "{GFPGAN_WHEEL_DIR}" {GFPGAN_PKGS} 2>/dev/null || true
    !pip install -q basicsr facexlib
    !pip install -q gfpgan realesrgan
    with open(GFPGAN_WHEEL_MARKER, 'w') as f:
        f.write('ok')

# 설치 확인
try:
    import gfpgan
    print('\u2705 GFPGAN 설치 확인 완료')
except ImportError as e:
    print(f'\u26a0\ufe0f GFPGAN import 실패: {e}')
    print('   pip install gfpgan으로 직접 설치를 시도합니다...')
    !pip install -q gfpgan

# 2) 모델 파일 (Drive에 영구 캐시)
GFPGAN_MODEL_CACHE = os.path.join(DRIVE_ROOT, 'BISTOOL_Models', 'GFPGANv1.4.pth')
GFPGAN_MODEL_DIR = os.path.join(WORK_DIR, 'gfpgan_models')
os.makedirs(GFPGAN_MODEL_DIR, exist_ok=True)
gfpgan_model_path = os.path.join(GFPGAN_MODEL_DIR, 'GFPGANv1.4.pth')

if os.path.exists(gfpgan_model_path):
    print(f'GFPGAN 모델 이미 로컬에 있음')
elif os.path.exists(GFPGAN_MODEL_CACHE):
    print('GFPGAN 모델 Drive 캐시에서 복사 중...')
    shutil.copy2(GFPGAN_MODEL_CACHE, gfpgan_model_path)
    print('GFPGAN 모델 복원 완료!')
else:
    print('GFPGAN 모델 다운로드 중...')
    !wget -q -O {gfpgan_model_path} https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth
    os.makedirs(os.path.dirname(GFPGAN_MODEL_CACHE), exist_ok=True)
    shutil.copy2(gfpgan_model_path, GFPGAN_MODEL_CACHE)
    print('GFPGAN 모델 다운로드 + Drive 캐시 저장 완료!')

print('\u2705 GFPGAN 준비 완료!')

In [ ]:
# ============================================================
# 6-3. CodeFormer로 얼굴 복원 실행
# \u2605 cd CodeFormer로 실행 → Python이 스크립트 디렉토리를 sys.path[0]에 추가
# \u2605 → 번들 basicsr과 facelib 자동 사용 (GFPGAN과 충돌 없음)
# ============================================================
CODEFORMER_RESULTS_DIR = os.path.join(RESULTS_DIR, 'CodeFormer')
os.makedirs(CODEFORMER_RESULTS_DIR, exist_ok=True)

# realesrgan 사용 가능한지 확인
try:
    import realesrgan
    _bg_upsampler = '--bg_upsampler realesrgan'
except ImportError:
    _bg_upsampler = ''
    print('  (realesrgan 미설치 -> 배경 업샘플링 건너뜀)')

# CodeFormer 추론 실행
codeformer_results = {}

# CT에서 추출한 이미지 + FaceScape 이미지 모두 처리
all_test_images = {}

# CT 이미지
if 'all_face_images' in dir() and all_face_images:
    for patient_id, paths in all_face_images.items():
        all_test_images[f'ct_{patient_id}'] = paths[0]

# FaceScape 이미지
if 'facescape_hrn_results' in dir() and facescape_hrn_results:
    for subject_id, info in facescape_hrn_results.items():
        if isinstance(info, dict) and 'error' not in info:
            all_test_images[subject_id] = info.get('input_image', '')

if not all_test_images:
    print('\u26a0\ufe0f 처리할 이미지가 없습니다. CT 데이터 또는 FaceScape 셀을 먼저 실행하세요.')
else:
    for img_id, img_path in all_test_images.items():
        output_path = os.path.join(CODEFORMER_RESULTS_DIR, f'{img_id}_codeformer.png')

        if os.path.exists(output_path):
            codeformer_results[img_id] = output_path
            continue

        print(f'CodeFormer 처리 중: {img_id}...')
        try:
            !cd {CODEFORMER_DIR} && python inference_codeformer.py \
                -w 0.7 \
                --input_path {img_path} \
                --output_path {os.path.join(CODEFORMER_RESULTS_DIR, img_id)} \
                {_bg_upsampler} \
                --face_upsample 2>/dev/null

            # 결과 파일 찾기
            result_dir = os.path.join(CODEFORMER_RESULTS_DIR, img_id, 'final_results')
            if os.path.exists(result_dir):
                result_files = glob.glob(os.path.join(result_dir, '*.png'))
                if result_files:
                    shutil.copy(result_files[0], output_path)
                    codeformer_results[img_id] = output_path
                    print(f'  완료: {output_path}')
                else:
                    print(f'  결과 파일 없음')
            else:
                print(f'  결과 디렉토리 없음')
        except Exception as e:
            print(f'  오류: {e}')

    print(f'\nCodeFormer 완료: {len(codeformer_results)} 이미지 처리됨')

In [ ]:
# ============================================================
# 6-4. GFPGAN으로 얼굴 복원 실행
# ============================================================
from gfpgan import GFPGANer

# GFPGAN 모델 로드
gfpganer = GFPGANer(
    model_path=gfpgan_model_path,
    upscale=2,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)

GFPGAN_RESULTS_DIR = os.path.join(RESULTS_DIR, 'GFPGAN')
os.makedirs(GFPGAN_RESULTS_DIR, exist_ok=True)

gfpgan_results = {}

if not all_test_images:
    print('\u26a0\ufe0f 처리할 이미지가 없습니다.')
else:
    for img_id, img_path in all_test_images.items():
        output_path = os.path.join(GFPGAN_RESULTS_DIR, f'{img_id}_gfpgan.png')

        if os.path.exists(output_path):
            gfpgan_results[img_id] = output_path
            continue

        print(f'GFPGAN 처리 중: {img_id}...')
        try:
            input_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if input_img is None:
                gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if gray is not None:
                    input_img = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
                else:
                    print(f'  이미지 로드 실패: {img_path}')
                    continue

            start_time = time.time()
            _, _, restored_img = gfpganer.enhance(
                input_img, has_aligned=False, only_center_face=False, paste_back=True
            )
            elapsed = time.time() - start_time

            cv2.imwrite(output_path, restored_img)
            gfpgan_results[img_id] = output_path
            print(f'  완료 ({elapsed:.2f}s): {output_path}')
        except Exception as e:
            print(f'  오류: {e}')

    print(f'\nGFPGAN 완료: {len(gfpgan_results)} 이미지 처리됨')

---
## 7. 정량적/정성적 결과 분석

### 평가 메트릭:
- **PSNR** (Peak Signal-to-Noise Ratio): 높을수록 좋음
- **SSIM** (Structural Similarity Index): 1에 가까울수록 좋음
- **LPIPS** (Learned Perceptual Image Patch Similarity): 낮을수록 좋음
- **추론 시간**: 실용성 비교

In [ ]:
# ============================================================
# 7-1. 정량적 평가 함수 정의
# ============================================================
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips
import pandas as pd

# LPIPS 모델 로드
lpips_model = lpips.LPIPS(net='alex').cuda()

def compute_metrics(img1_path, img2_path):
    """두 이미지 간의 PSNR, SSIM, LPIPS를 계산합니다."""
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    if img1 is None or img2 is None:
        return {'psnr': None, 'ssim': None, 'lpips': None}

    # 크기 맞추기
    h = min(img1.shape[0], img2.shape[0])
    w = min(img1.shape[1], img2.shape[1])
    img1 = cv2.resize(img1, (w, h))
    img2 = cv2.resize(img2, (w, h))

    # PSNR
    psnr_val = psnr(img1, img2)

    # SSIM
    ssim_val = ssim(img1, img2, channel_axis=2)

    # LPIPS
    t1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).float() / 127.5 - 1
    t2 = torch.from_numpy(img2).permute(2, 0, 1).unsqueeze(0).float() / 127.5 - 1
    with torch.no_grad():
        lpips_val = lpips_model(t1.cuda(), t2.cuda()).item()

    return {'psnr': psnr_val, 'ssim': ssim_val, 'lpips': lpips_val}

print('평가 메트릭 함수 준비 완료')

In [ ]:
# ============================================================
# 7-2. 모든 방법의 결과를 원본 이미지와 비교
# ============================================================
comparison_data = []

for img_id, original_path in all_test_images.items():
    row = {'image_id': img_id}

    # HRN 시각화 결과와 비교 (2D rendered face)
    hrn_info = hrn_results.get(img_id.replace('ct_', ''), facescape_hrn_results.get(img_id, {}))
    hrn_vis_path = hrn_info.get('saved_files', {}).get('visualization') if isinstance(hrn_info, dict) else None

    if hrn_vis_path and os.path.exists(hrn_vis_path):
        metrics = compute_metrics(original_path, hrn_vis_path)
        row['hrn_psnr'] = metrics['psnr']
        row['hrn_ssim'] = metrics['ssim']
        row['hrn_lpips'] = metrics['lpips']
        row['hrn_time'] = hrn_info.get('inference_time', None)

    # CodeFormer 결과와 비교
    cf_path = codeformer_results.get(img_id)
    if cf_path and os.path.exists(cf_path):
        metrics = compute_metrics(original_path, cf_path)
        row['codeformer_psnr'] = metrics['psnr']
        row['codeformer_ssim'] = metrics['ssim']
        row['codeformer_lpips'] = metrics['lpips']

    # GFPGAN 결과와 비교
    gf_path = gfpgan_results.get(img_id)
    if gf_path and os.path.exists(gf_path):
        metrics = compute_metrics(original_path, gf_path)
        row['gfpgan_psnr'] = metrics['psnr']
        row['gfpgan_ssim'] = metrics['ssim']
        row['gfpgan_lpips'] = metrics['lpips']

    comparison_data.append(row)

df = pd.DataFrame(comparison_data)
print('=== 정량적 비교 결과 ===')
print(df.to_string(index=False))
df.to_csv(os.path.join(RESULTS_DIR, 'Comparison', 'quantitative_results.csv'), index=False)
print(f'\n저장: {os.path.join(RESULTS_DIR, "Comparison", "quantitative_results.csv")}')

In [ ]:
# ============================================================
# 7-3. 평균 메트릭 요약 테이블
# ============================================================
summary_metrics = {}

for method in ['hrn', 'codeformer', 'gfpgan']:
    psnr_col = f'{method}_psnr'
    ssim_col = f'{method}_ssim'
    lpips_col = f'{method}_lpips'

    if psnr_col in df.columns:
        summary_metrics[method.upper()] = {
            'PSNR (dB) ↑': df[psnr_col].dropna().mean(),
            'SSIM ↑': df[ssim_col].dropna().mean(),
            'LPIPS ↓': df[lpips_col].dropna().mean(),
        }

summary_df = pd.DataFrame(summary_metrics).T
print('=' * 60)
print('          평균 메트릭 비교 요약')
print('=' * 60)
print(summary_df.to_string())
print('=' * 60)
print('↑ = 높을수록 좋음, ↓ = 낮을수록 좋음')

summary_df.to_csv(os.path.join(RESULTS_DIR, 'Comparison', 'summary_metrics.csv'))
print(f'\n저장: {os.path.join(RESULTS_DIR, "Comparison", "summary_metrics.csv")}')

In [ ]:
# ============================================================
# 7-4. 시각적 비교 (정성적 평가)
# ============================================================
# CT 데이터에 대한 비교 시각화
ct_image_ids = [k for k in all_test_images.keys() if k.startswith('ct_')]

n_display = min(len(ct_image_ids), 6)
if n_display == 0:
    print('표시할 CT 이미지가 없습니다.')
else:
    fig, axes = plt.subplots(n_display, 4, figsize=(20, 5 * n_display))
    if n_display == 1:
        axes = axes[np.newaxis, :]

    col_titles = ['Original (CT)', 'HRN (3D Recon)', 'CodeFormer', 'GFPGAN']

    for i, img_id in enumerate(ct_image_ids[:n_display]):
        original_path = all_test_images[img_id]

        # Original
        orig = cv2.imread(original_path)
        if orig is not None:
            axes[i, 0].imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB) if len(orig.shape) == 3 else orig, cmap='gray')
        axes[i, 0].set_title(f'{col_titles[0]}\n{img_id}')
        axes[i, 0].axis('off')

        # HRN
        patient_id = img_id.replace('ct_', '')
        hrn_info = hrn_results.get(patient_id, {})
        vis_path = hrn_info.get('saved_files', {}).get('visualization') if isinstance(hrn_info, dict) else None
        if vis_path and os.path.exists(vis_path):
            hrn_img = cv2.imread(vis_path)
            axes[i, 1].imshow(cv2.cvtColor(hrn_img, cv2.COLOR_BGR2RGB))
        else:
            axes[i, 1].text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
        axes[i, 1].set_title(col_titles[1])
        axes[i, 1].axis('off')

        # CodeFormer
        cf_path = codeformer_results.get(img_id)
        if cf_path and os.path.exists(cf_path):
            cf_img = cv2.imread(cf_path)
            axes[i, 2].imshow(cv2.cvtColor(cf_img, cv2.COLOR_BGR2RGB))
        else:
            axes[i, 2].text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
        axes[i, 2].set_title(col_titles[2])
        axes[i, 2].axis('off')

        # GFPGAN
        gf_path = gfpgan_results.get(img_id)
        if gf_path and os.path.exists(gf_path):
            gf_img = cv2.imread(gf_path)
            axes[i, 3].imshow(cv2.cvtColor(gf_img, cv2.COLOR_BGR2RGB))
        else:
            axes[i, 3].text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
        axes[i, 3].set_title(col_titles[3])
        axes[i, 3].axis('off')

    plt.suptitle('얼굴 복원 방법 비교 (CT 데이터)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'Comparison', 'visual_comparison_ct.png'), dpi=200, bbox_inches='tight')
    plt.show()

In [ ]:
# ============================================================
# 7-5. 메트릭 비교 차트
# ============================================================
if summary_df.empty:
    print('비교할 데이터가 없습니다.')
else:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    methods = list(summary_df.index)
    colors = ['#2196F3', '#FF9800', '#4CAF50']

    # PSNR 비교
    psnr_vals = summary_df['PSNR (dB) ↑'].values
    axes[0].bar(methods, psnr_vals, color=colors)
    axes[0].set_title('PSNR (dB) ↑', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('dB')
    for j, v in enumerate(psnr_vals):
        if not np.isnan(v):
            axes[0].text(j, v + 0.3, f'{v:.2f}', ha='center', fontweight='bold')

    # SSIM 비교
    ssim_vals = summary_df['SSIM ↑'].values
    axes[1].bar(methods, ssim_vals, color=colors)
    axes[1].set_title('SSIM ↑', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Score')
    for j, v in enumerate(ssim_vals):
        if not np.isnan(v):
            axes[1].text(j, v + 0.005, f'{v:.4f}', ha='center', fontweight='bold')

    # LPIPS 비교
    lpips_vals = summary_df['LPIPS ↓'].values
    axes[2].bar(methods, lpips_vals, color=colors)
    axes[2].set_title('LPIPS ↓', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Distance')
    for j, v in enumerate(lpips_vals):
        if not np.isnan(v):
            axes[2].text(j, v + 0.005, f'{v:.4f}', ha='center', fontweight='bold')

    plt.suptitle('Face Restoration Methods - Quantitative Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'Comparison', 'metrics_comparison_chart.png'), dpi=200, bbox_inches='tight')
    plt.show()

---
## 8. 결론 및 분석

### 실험 요약

| 방법 | 유형 | 핵심 기술 | 장점 | 단점 |
|------|------|-----------|------|------|
| **HRN** (논문) | 2D→3D 복원 | 계층적 표현 (Low/Mid/High freq) + 3DMM | 고품질 3D mesh, 디테일 분리 | 정면 얼굴 이미지 필요 |
| **CodeFormer** | 2D→2D 복원 | Codebook + Transformer | 자연스러운 복원, fidelity 조절 가능 | 3D 정보 없음 |
| **GFPGAN** | 2D→2D 복원 | GAN + Facial Prior | 빠른 추론 속도 | 과도한 smoothing 가능 |

### 핵심 발견
1. **HRN**은 단일 2D 이미지에서 mid/high-frequency 디테일을 포함한 정확한 3D 얼굴 모델을 생성할 수 있음
2. **CodeFormer/GFPGAN**은 2D 이미지 품질 향상에 효과적이나, 3D 기하학적 정보는 제공하지 않음
3. CT 데이터에서 추출한 2D 투영 이미지는 일반 사진과 다른 특성을 가져 모든 방법에서 추가 전처리가 필요할 수 있음
4. FaceScape 데이터에서는 HRN이 논문의 벤치마크 결과와 일치하는 성능을 보여줌

### 결과 저장 위치
모든 결과는 Google Drive의 `BISTOOL_Results/` 폴더에 저장되었습니다:
- `HRN/`: 3D mesh (.obj), 텍스처, 시각화
- `CodeFormer/`: 복원된 2D 이미지
- `GFPGAN/`: 복원된 2D 이미지
- `Comparison/`: 정량적 비교 CSV, 시각적 비교 이미지

In [ ]:
# ============================================================
# 8. CT 3D 표면 vs HRN 3D 복원 비교 (추가 분석)
# ============================================================
from scipy.spatial import cKDTree

def chamfer_distance(verts1, verts2, num_samples=10000):
    """두 메쉬 간의 Chamfer Distance를 계산합니다."""
    idx1 = np.random.choice(len(verts1), min(num_samples, len(verts1)), replace=False)
    idx2 = np.random.choice(len(verts2), min(num_samples, len(verts2)), replace=False)
    v1, v2 = verts1[idx1], verts2[idx2]
    tree1, tree2 = cKDTree(v1), cKDTree(v2)
    dist1, _ = tree2.query(v1)
    dist2, _ = tree1.query(v2)
    return np.mean(dist1) + np.mean(dist2)

print('CT 3D 표면 vs HRN 3D 복원 메쉬 기하학적 비교')
print('=' * 60)

geo_comparison = []
for patient_id in ct_folders.keys():
    hrn_info = hrn_results.get(patient_id, {})
    mesh_path = hrn_info.get('saved_files', {}).get('mesh') if isinstance(hrn_info, dict) else None

    if mesh_path and os.path.exists(mesh_path):
        try:
            hrn_mesh = trimesh.load(mesh_path)
            # ★ ct_folders는 이미 로컬 경로를 가리킴 (빠름)
            vol, sp, _ = load_dicom_series(ct_folders[patient_id]['path'])
            ct_verts, ct_faces, _ = extract_face_surface(vol, sp, threshold=200)
            del vol
            cd = chamfer_distance(ct_verts, hrn_mesh.vertices)
            geo_comparison.append({
                'patient_id': patient_id,
                'ct_vertices': ct_verts.shape[0],
                'hrn_vertices': hrn_mesh.vertices.shape[0],
                'chamfer_distance': cd,
            })
            print(f'  Patient {patient_id}: CD = {cd:.4f} mm')
        except Exception as e:
            print(f'  Patient {patient_id}: 오류 - {e}')

if geo_comparison:
    geo_df = pd.DataFrame(geo_comparison)
    print(f'\n평균 Chamfer Distance: {geo_df["chamfer_distance"].mean():.4f} mm')
    geo_df.to_csv(os.path.join(RESULTS_DIR, 'Comparison', 'geometric_comparison.csv'), index=False)
else:
    print('기하학적 비교 가능한 데이터 없음')

In [ ]:
# ============================================================
# 최종 결과 요약 출력
# ============================================================
print('=' * 70)
print('          HRN Face Reconstruction - 실험 완료')
print('=' * 70)
print()
print(f'처리된 CT 환자 수: {len(ct_folders)}')
print(f'HRN 성공 건수: {len([v for v in hrn_results.values() if isinstance(v, dict) and "error" not in v])}')
print(f'CodeFormer 성공 건수: {len(codeformer_results)}')
print(f'GFPGAN 성공 건수: {len(gfpgan_results)}')
print()
print(f'결과 저장 위치: {RESULTS_DIR}')
print()
print('저장된 결과물:')
for root, dirs, files in os.walk(RESULTS_DIR):
    for f in files:
        rel_path = os.path.relpath(os.path.join(root, f), RESULTS_DIR)
        size_kb = os.path.getsize(os.path.join(root, f)) / 1024
        print(f'  {rel_path}: {size_kb:.1f} KB')
print()
print('실험 완료!')